# Data Management
This notebook includes code for managing data, including generating representative sets of testing sites, running models on these testing sites, and evaluating the qualities/quantities of training and testing sets. The code is not meant to be run together; each section should be run independently when required. 

In [1]:
# preliminaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import random as rn
import pickle
import traceback
import torch
import hf_hydrodata as hf
from tqdm import tqdm

from _lstm import *
from _data import *

import io
from contextlib import redirect_stdout
trap = io.StringIO()

# Numbers of sites

In [ ]:
## TOTAL NUMBER OF SNOTEL SITES ##
snotel_full = hf.get_site_variables(variable="swe")
print(len(snotel_full))

In [17]:
## NUMBER OF SITES IN TRAINING DATA ##
run='l6_200_m40'
test = pd.read_csv('Data/LSTM_output/'+run+'_train_metadata.csv', sep=' ')

len(np.unique(test['site_id']))

198

# Generate representative set of test sites for western US

In [ ]:
# ## GENERATE TESTING DATA ##
# # GET TESTING SITES
# test = get_sites_latitude(120)
# test = test.dropna(axis='index', how='any').reset_index().drop(columns=['index'])

# test.head()
# test.to_csv('national_test_sites.txt', sep=' ',header=None, index=False, index_label=False)

# # GET TESTING YEARS
# data_test = pd.read_csv('national_test_sites.txt', sep=' ',header=None)
# data_test.columns = ['site_id', 'site_name', 'site_type', 'agency', 'state','first_date_data_available', 'last_date_data_available', 'record_count',
#                      'latitude', 'longitude', 'bins','first_wy_date', 'last_wy_date']
# #data_test = data_test.drop(columns=['bins', 'first_wy_date', 'last_wy_date'])
# data_test

# data = pd.DataFrame(columns=['site_id', 'year','train'])

# for i in range(0, len(data_test)):
#     site_id = data_test['site_id'][i]
#     start_date = data_test['first_wy_date'][i]
#     end_date = data_test['last_wy_date'][i]
#     try:
#         with redirect_stdout(trap):
#             years = get_years_precip(start_date, end_date, site_id, 3) 

#         for j in range(0, len(years)):
#             data.loc[len(data.index)] = [site_id, int(years[j]), False] 
#     except:
#         print('missing data for ', site_id)
#         traceback.print_exc()

# data.head()
# data.to_csv('national_test_years.txt', sep=' ',header=None, index=False, index_label=False)

In [ ]:
## GENERATE NEW TESTING DATA FOR EACH MODEL ##
RUN_name = ['l6_500']#,'l3_200','l5_200']
#['p5_36','p10_36','p3_72','p5_72','l3_36','l5_36','l10_36','l3_72','l5_72']

for run_name in RUN_name:
    l_swe_test = [] 
    l_non_swe_test = []
    
    data_test = pd.read_csv('national_test_years.txt', sep=' ',header=None)
    data_test.columns = ['site_id',	'year',	'train']
    
    # GET TESTING DATA
    for j in range(0, len(data_test)):
        site_id = data_test['site_id'][j]
        year = data_test['year'][j]
        start_date = str(year-1) + '-10-01'
        end_date = str(year) + '-09-30'
        try:
            with redirect_stdout(trap):
                swe, non_swe = get_sc_data(site_id, start_date, end_date)
            l_swe_test.append(swe)
            l_non_swe_test.append(non_swe)
            # if j == 0:
            #     plt.plot(swe['swe'],label='from hydrodata by way of code')
    
            # add site data
            data_test.loc[j, 'latitude'] = non_swe.loc[0,'latitude']
            data_test.loc[j, 'longitude'] = non_swe.loc[0,'longitude']
            data_test.loc[j, 'elevation'] = non_swe.loc[0,'elevation']
            data_test.loc[j, 'land cover'] = non_swe.loc[0,'land_cover']
            data_test.loc[j, 'slope_x'] = non_swe.loc[0,'slope_x']
            data_test.loc[j, 'slope_y'] = non_swe.loc[0,'slope_y']
        except:
            print('missing data for ', site_id, " : ", year)
            traceback.print_exc()
    
    print('got data')
    
    test_swe = pd.concat(l_swe_test).reset_index().drop(columns='index')
    test_non_swe = pd.concat(l_non_swe_test).reset_index().drop(columns='index')
    #plt.plot(test_swe['swe'][0:365])
    with open('output/'+run_name + '_normalize.pkl', 'rb') as file:  
        l_normalize = pickle.load(file)
    scaler_swe = l_normalize[0]
    test_swe_tensors, test_non_swe_tensors, test_sites, test_years = create_dataset(test_swe, test_non_swe, l_normalize)
    #plt.plot(scaler_swe.inverse_transform(test_swe_tensors[0].detach().numpy()))
    #torch.save(test_swe_tensors, 'output/'+run_name+'_test_swe.pt')
    #torch.save(test_non_swe_tensors, 'output/'+run_name+'_test_non_swe.pt')
    #data_test.to_csv('output/test_metadata.csv', sep=' ',header=None, index=False, index_label=False)

In [ ]:
## SAVE METADATA FOR TESTING SITES AND YEARS ##
# test_swe and test_non_swe contain all information about test data
data_test = pd.read_csv('national_test_years.txt', sep=' ',header=None)
data_test.columns = ['site_id',	'year',	'train']

l_swe_test = [] 
l_non_swe_test = []

# GET TESTING DATA
count_missing = 0
for j in tqdm(range(0, len(data_test))):
    site_id = data_test['site_id'][j]
    year = data_test['year'][j]
    start_date = str(year-1) + '-10-01'
    end_date = str(year) + '-09-30'
    try:
        with redirect_stdout(trap):
            swe, non_swe = get_sc_data(site_id, start_date, end_date)
        l_swe_test.append(swe)
        l_non_swe_test.append(non_swe)

        # add site data
        data_test.loc[j, 'latitude'] = non_swe.loc[0,'latitude']
        data_test.loc[j, 'longitude'] = non_swe.loc[0,'longitude']
        data_test.loc[j, 'elevation'] = non_swe.loc[0,'elevation']
        data_test.loc[j, 'land cover'] = non_swe.loc[0,'land_cover']
        data_test.loc[j, 'slope_x'] = non_swe.loc[0,'slope_x']
        data_test.loc[j, 'slope_y'] = non_swe.loc[0,'slope_y']
    except:
        #print('missing data for ', site_id, " : ", year)
        data_test.loc[j, 'missing'] = True
        #traceback.print_exc()
        count_missing += 1
        
print('number of missing training/testing sites: ',count_missing,' out of ',len(data_test),' sites')
    
test_swe = pd.concat(l_swe_test).reset_index().drop(columns='index')
test_non_swe = pd.concat(l_non_swe_test).reset_index().drop(columns='index')
test_swe.to_pickle('output/test_swe.pkl')
test_non_swe.to_pickle('output/test_non_swe.pkl')

#data_test.to_csv('output/test_metadata.csv', sep=' ',header=None, index=False, index_label=False)

In [2]:
## RUN OLD MODELS ON NEW TESTING DATA ## 
RUN_name = ['l9_500']

# open data
data_test = pd.read_csv('national_test_years.txt',sep=' ',header=None)
data_test.columns = ['site_id',	'year',	'train']
with open('Data/LSTM_output/test_swe.pkl', 'rb') as file:  
    test_swe = pickle.load(file)
with open('Data/LSTM_output/test_non_swe.pkl', 'rb') as file:  
    test_non_swe = pickle.load(file)
#test_swe = torch.load('/home/mcburns/national_lstm/output/'+run+'_test_swe.pt')
#test_non_swe = torch.load('/home/mcburns/national_lstm/output/'+run+'_test_non_swe.pt')

for k in range(0, len(RUN_name)):
    ## DEFINE RUN 
    run = RUN_name[k]

    # check if any sites overlap
    metadata = pd.read_csv('Data/LSTM_output/'+run+'_train_metadata.csv',sep=' ')
    metadata['check'] = (metadata['site_id'].isin(data_test['site_id']) & metadata['year'].isin(data_test['year']))
    metadata[metadata['check']]
    print("total number of shared sites for ", run," : ", metadata['check'].sum(), " out of ", len(metadata))

    ## GET DATA
    with open('/home/mcburns/national_lstm/Data/LSTM_output/'+run+'_normalize.pkl', 'rb') as file:  
        l_normalize = pickle.load(file)
    scaler_swe = l_normalize[0]
    test_swe_tensors, test_non_swe_tensors, test_sites, test_years = create_dataset(test_swe, test_non_swe, l_normalize)
    
    ## EVALUATE LSTM
    ev_lstm = torch.load('Data/LSTM_output/'+run+'_lstm.pt', map_location = DEVICE)
    
    statistics, feature_importance = analyze_results_lstm(ev_lstm, data_test, test_swe_tensors, test_non_swe_tensors, scaler_swe, False)

    ## SAVE DATA
    feature_importance = pd.DataFrame(feature_importance)
    feature_importance.to_csv('Data/LSTM_output/'+run+'_features.txt',sep=' ',header=None, index=False, index_label=False)
    statistics.to_csv('Data/LSTM_output/'+run+'_statistics.txt',sep=' ',header=None, index=False, index_label=False)
    
    print('statistics for: ' + run)
    print(f"RMSE: {np.mean(statistics['rmse']):.2f}")
    print(f"normal RMSE: {np.mean(statistics['normal rmse']):.2f}")
    print(f"NSE: {np.mean(statistics['nse']):.2f}")
    print(f"R2: {np.mean(statistics['r2']):.2f}")
    print(f"Spearman's rho: {np.mean(statistics['spearman_rho']):.2f}")
    print(f"delta peak SWE: {np.mean(statistics['delta peak']):.2f}")
    print(f"normal delta peak SWE: {np.mean(statistics['normal delta peak']):.2f}")
    print(f"absolute delta peak SWE: {np.mean(statistics['abs delta peak']):.2f}")
    print(f"normal absolute delta peak SWE: {np.mean(statistics['normal abs delta peak']):.2f}")
    print(f"delta days: {np.mean(statistics['delta days']):.2f}")
    print(f"absolute delta days: {np.mean(statistics['abs delta days']):.2f}")

total number of shared sites for  l9_500  :  0  out of  4284


/tmp/ipykernel_2446841/1635139469.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ev_lstm = torch.load('Data/LSTM_output/'+run+'_lstm.pt', map_location = DEVICE)


statistics for: l9_500
RMSE: 64.43
normal RMSE: 0.16
NSE: 0.57
R2: 0.57
Spearman's rho: 0.88
delta peak SWE: -46.65
normal delta peak SWE: -0.09
absolute delta peak SWE: 95.92
normal absolute delta peak SWE: 0.25
delta days: 4.43
absolute delta days: 13.58


# Verify that training and testing sites do not overlap

In [ ]:
run = 'l9_500'

test_sites = pd.read_csv('national_test_sites.txt',sep=' ',header=None)
test_sites.columns=['site_id','name','type','agency','state','start date','end date','number of available sites','latitude','longitude','?','wy_start','wy_end']
test_years = pd.read_csv('national_test_years.txt',sep=' ',header=None)
test_years.columns=['site_id','year','train']

metadata = pd.read_csv('output/'+run+'_train_metadata.csv',sep=' ')


metadata['check'] = (metadata['site_id'].isin(test_years['site_id']) & metadata['year'].isin(test_years['year']))


print("total number of shared sites: ", metadata['check'].sum(), " out of ", len(metadata))

## other checks
# np.unique(metadata[metadata['check']]['site_id'])
# metadata['check'] = metadata['site_id'].isin(test_sites['site_id']) 
# metadata[metadata['check']]

# Number of sites in each snowpack regime

In [2]:
## GET DATA ##
snotel = hf.get_site_variables(variable="swe")
snotel = snotel.reset_index(drop=True).drop(columns=['variable_name','units','site_query_url','date_metadata_last_updated','tz_cd','doi'])

# read in testing data and remove it from dataset
# data_test = pd.read_csv('national_test_sites.txt', sep=' ',header=None)
# data_test.columns = ['site_id', 'site_name', 'site_type', 'agency', 'state','first_date_data_available', 'last_date_data_available', 'record_count',
#                      'latitude', 'longitude', 'bins', 'first_wy_date', 'last_wy_date']
# data_test = data_test.drop(columns=['bins'])#, 'first_wy_date', 'last_wy_date'])
# snotel = snotel[~snotel['site_id'].isin(data_test['site_id'])].reset_index(drop=True)

snotel['first_wy_date'] = snotel.apply(lambda x: get_wy_start(pd.to_datetime(x.first_date_data_available), x.site_id), axis=1)
snotel['last_wy_date'] = snotel.apply(lambda x: get_wy_end(pd.to_datetime(x.last_date_data_available), x.site_id), axis=1)
snotel = snotel.dropna(axis='index', subset=['first_wy_date','last_wy_date'], how='any').reset_index(drop=True)

snotel['num years'] = np.array(list(int(x.split('-')[0]) for x in snotel['last_wy_date'])) - np.array(list(int(x.split('-')[0]) for x in snotel['first_wy_date']))
#snotel = snotel[snotel['num years'] >= 3].reset_index(drop=True)

snotel

,site_id,site_name,site_type,agency,state,dataset,variable,temporal_resolution,aggregation,first_date_data_available,...,record_count,latitude,longitude,conus1_i,conus1_j,conus2_i,conus2_j,first_wy_date,last_wy_date,num years
0,301:CA:SNTL,Adin Mtn,SNOTEL station,NRCS,CA,usda_nrcs,swe,daily,sod,1984-10-01,...,14643,41.23583,-120.79192,NaN,NaN,309.0,2086.0,1990-10-01,2022-09-30,32
1,907:UT:SNTL,Agua Canyon,SNOTEL station,NRCS,UT,usda_nrcs,swe,daily,sod,1994-10-01,...,10991,37.52217,-112.27118,461.0,569.0,902.0,1525.0,2006-10-01,2022-09-30,16
2,916:MT:SNTL,Albro Lake,SNOTEL station,NRCS,MT,usda_nrcs,swe,daily,sod,1996-10-01,...,10260,45.59723,-111.95902,644.0,1444.0,1090.0,2375.0,1996-10-01,2022-09-30,26
3,908:WA:SNTL,Alpine Meadows,SNOTEL station,NRCS,WA,usda_nrcs,swe,daily,sod,1994-10-01,...,10991,47.77957,-121.69847,NaN,NaN,453.0,2778.0,1994-10-01,2022-09-30,28
4,302:OR:SNTL,Aneroid Lake #2,SNOTEL station,NRCS,OR,usda_nrcs,swe,daily,sod,1982-10-01,...,15374,45.21328,-117.19258,234.0,1485.0,696.0,2421.0,1984-10-01,2022-09-30,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,1228:UT:SNTL,Wrigley Creek,SNOTEL station,NRCS,UT,usda_nrcs,swe,daily,sod,2012-10-01,...,4414,39.13233,-111.35685,570.0,730.0,1011.0,1682.0,2012-10-01,2022-09-30,10
808,1197:UT:SNTL,Yankee Reservoir,SNOTEL station,NRCS,UT,usda_nrcs,swe,daily,sod,2012-10-01,...,4416,37.74797,-112.77495,422.0,602.0,865.0,1558.0,2012-10-01,2022-09-30,10
809,878:WY:SNTL,Younts Peak,SNOTEL station,NRCS,WY,usda_nrcs,swe,daily,sod,1980-09-02,...,16133,43.93225,-109.81775,781.0,1234.0,1220.0,2171.0,1988-10-01,2022-09-30,34
810,1033:CO:SNTL,Zirkel,SNOTEL station,NRCS,CO,usda_nrcs,swe,daily,sod,2002-10-01,...,8069,40.79492,-106.59544,997.0,855.0,1427.0,1801.0,2002-10-01,2022-09-30,20


In [8]:
np.unique(snotel['state'])

array(['AZ', 'CA', 'CO', 'ID', 'MT', 'NM', 'NV', 'OR', 'SD', 'UT', 'WA',
       'WY'], dtype=object)

In [ ]:
## ? ##
# not really sure what's going on in this code
len(snotel[snotel['state'] == 'OR'])

# bin snotel sites into 3 based on longitude
num_sites = 200
bins = [-125, -118, -111, -100]
bin_labels=['maritime','intermountain','continental'] 
test = snotel
test['bins'] = pd.cut(test['longitude'], bins=bins, labels=bin_labels, include_lowest=True)
test
print('intermountain: ', len(test[test['bins'] == 'intermountain']))
print('continental: ', len(test[test['bins'] == 'continental']))
print('maritime: ', len(test[test['bins'] == 'maritime']))

pd.set_option('display.max_rows', None)    # To display all rows
pd.set_option('display.max_columns', None) # To display all columns
snotel[snotel['state'] == 'CA']